### Import Files

In [1]:
import pandas as pd
file_1=pd.read_excel(r"C:\Users\Parimal\Downloads\Cointab Data Analyst - Challenge\Company X - Order Report.xlsx")
file_2=pd.read_excel(r"C:\Users\Parimal\Downloads\Cointab Data Analyst - Challenge\Company X - Pincode Zones.xlsx")
file_3=pd.read_excel(r"C:\Users\Parimal\Downloads\Cointab Data Analyst - Challenge\Company X - SKU Master.xlsx")
file_4=pd.read_excel(r"C:\Users\Parimal\Downloads\Cointab Data Analyst - Challenge\Courier Company - Invoice.xlsx")
file_5=pd.read_excel(r"C:\Users\Parimal\Downloads\Cointab Data Analyst - Challenge\Courier Company - Rates.xlsx")
file_out_1=pd.read_excel(r"C:\Users\Parimal\Downloads\Cointab Data Analyst - Challenge\Expected_Result.xlsx",sheet_name='Summary')
file_out_2=pd.read_excel(r"C:\Users\Parimal\Downloads\Cointab Data Analyst - Challenge\Expected_Result.xlsx",sheet_name='Calculations')

### All Uique Order ID's
(ID'd are in asscending order)

In [2]:
import numpy as np
import pandas as pd

ID=[i for i in file_1['ExternOrderNo'].unique()]
ID.sort(reverse=False)
ID

file_output=pd.DataFrame(ID,columns={'Order ID'})
print(file_output.shape)

(124, 1)


### Handiling duplicates in data

In [3]:
file_1=file_1.drop_duplicates(ignore_index=True)
file_2=file_2.drop_duplicates(ignore_index=True)
file_3=file_3.drop_duplicates(ignore_index=True)

### Ouput_Data_1

In [4]:
AWB=[]
TW_X=[]
SLAB_X=[]
TW_C=[]
SLAB_C=[]
ZONE_X=[]
ZONE_C=[]
CHRG_X=[]
CHRG_C=[]
DIFF=[]


for i in range(file_output.shape[0]):
    ID=file_output['Order ID'][i]
    
    # AWB number
    i_4=np.where(file_4['Order ID']==ID)
    i_4=i_4[0][0]
    awb=file_4['AWB Code'][i_4]
    AWB.append(awb)
    
    # total weight of order as per x
    i_1=np.where(file_1['ExternOrderNo']==ID)
    i_1=i_1[0]  #bunch of index of similar oderID
    skw=0
    for j in i_1:
        sk=file_1['SKU'][j]
        qt=file_1['Order Qty'][j]
        i_3=np.where(file_3['SKU']==sk)
        i_3=i_3[0][0]
        sk_w=file_3['Weight (g)'][i_3]
        sk_w=sk_w/1000
        sk_w=sk_w*qt #### sku (product) induviudal weight mutiplied by quentity sku
        sk_w=skw+sk_w
        skw=sk_w
    TW_X.append(skw)
    
    # weight slab as per x
    inte=skw//1
    fra=skw%1
    if fra==0:
        sl_x=inte
        SLAB_X.append(sl_x)
    elif fra>0 and fra<=0.5:
        sl_x=inte+0.5
        SLAB_X.append(inte+0.5)
    else:
        sl_x=inte+1
        SLAB_X.append(inte+1)
    
    # total weight of order as per c
    tw_c=file_4['Charged Weight'][i_4]
    TW_C.append(tw_c)
    
    # weight slab as per c
    inte=tw_c//1
    fra=tw_c%1
    if fra==0:
        sl_c=inte
        SLAB_C.append(sl_c)
    elif fra>0 and fra<=0.5:
        sl_c=inte+0.5
        SLAB_C.append(sl_c)
    else:
        sl_c=inte+1
        SLAB_C.append(sl_c)
        
    # Delivery zone as per x
    pin_x=file_4['Customer Pincode'][i_4]
    i_2=np.where(file_2['Customer Pincode']==pin_x)
    i_2=i_2[0][0]
    z_x=file_2['Zone'][i_2]
    z_x=z_x.upper()
    ZONE_X.append(z_x)
    
    #Delivery zone as per c
    z_c=file_4['Zone'][i_4]
    z_c=z_c.upper()
    ZONE_C.append(z_c)
    
    #Expected Charge as per X (Rs.)
    ts=file_4['Type of Shipment'][i_4] ##Type of Shipment
    if ts=='Forward charges':
        if z_x=='A':
            if sl_x<=0.5:
                chrg=file_5['fwd_a_fixed'][0]
                CHRG_X.append(chrg)
            else:
                no_sl=sl_x*2
                chrg=file_5['fwd_a_fixed'][0]+((no_sl-1)*(file_5['fwd_a_additional'][0]))
                CHRG_X.append(chrg)
        elif z_x=='B':
            if sl_x<=0.5:
                chrg=file_5['fwd_b_fixed'][0] ###########################
                CHRG_X.append(chrg)
            else:
                no_sl=sl_x*2
                chrg=file_5['fwd_b_fixed'][0]+((no_sl-1)*(file_5['fwd_b_additional'][0]))
                CHRG_X.append(chrg)
        elif z_x=='C':
            if sl_x<=0.5:
                chrg=file_5['fwd_c_fixed'][0]
                CHRG_X.append(chrg)
            else:
                no_sl=sl_x*2
                chrg=file_5['fwd_c_fixed'][0]+((no_sl-1)*(file_5['fwd_c_additional'][0]))
                CHRG_X.append(chrg)
        elif z_x=='D':
            if sl_x<=0.5:
                chrg=file_5['fwd_d_fixed'][0]
                CHRG_X.append(chrg)
            else:
                no_sl=sl_x*2
                chrg=file_5['fwd_d_fixed'][0]+((no_sl-1)*(file_5['fwd_d_additional'][0]))
                CHRG_X.append(chrg)
        else:
            if sl_x<=0.5:
                chrg=file_5['fwd_e_fixed'][0]
                CHRG_X.append(chrg)
            else:
                no_sl=sl_x*2
                chrg=file_5['fwd_e_fixed'][0]+((no_sl-1)*(file_5['fwd_e_additional'][0]))
                CHRG_X.append(chrg)
    
    else:
        if z_x=='A':
            if sl_x<=0.5:
                chrg=file_5['fwd_a_fixed'][0]+file_5['rto_a_fixed'][0]
                CHRG_X.append(chrg)
            else:
                no_sl=sl_x*2
                chrg=(file_5['fwd_a_fixed'][0]+file_5['rto_a_fixed'][0])+((no_sl-1)*(file_5['fwd_a_additional'][0]+file_5['rto_a_additional'][0]))
                CHRG_X.append(chrg)
        elif z_x=='B':
            if sl_x<=0.5:
                chrg=file_5['fwd_b_fixed'][0]+file_5['rto_b_fixed'][0]
                CHRG_X.append(chrg)
            else:
                no_sl=sl_x*2
                chrg=(file_5['fwd_b_fixed'][0]+file_5['rto_b_fixed'][0])+((no_sl-1)*(file_5['fwd_b_additional'][0]+file_5['rto_b_additional'][0]))
                CHRG_X.append(chrg)
        elif z_x=='C':
            if sl_x<=0.5:
                chrg=file_5['fwd_c_fixed'][0]+file_5['rto_c_fixed'][0]
                CHRG_X.append(chrg)
            else:
                no_sl=sl_x*2
                chrg=(file_5['fwd_c_fixed'][0]+file_5['rto_c_fixed'][0])+((no_sl-1)*(file_5['fwd_c_additional'][0]+file_5['rto_c_additional'][0]))
                CHRG_X.append(chrg)
        elif z_x=='D':
            if sl_x<=0.5:
                chrg=file_5['fwd_d_fixed'][0]+file_5['rto_d_fixed'][0]
                CHRG_X.append(chrg)
            else:
                no_sl=sl_x*2
                chrg=(file_5['fwd_d_fixed'][0]+file_5['rto_d_fixed'][0])+((no_sl-1)*(file_5['fwd_d_additional'][0]+file_5['rto_d_additional'][0]))
                CHRG_X.append(chrg)
        else:
            if sl_x<=0.5:
                chrg=file_5['fwd_e_fixed'][0]+file_5['rto_e_fixed'][0]
                CHRG_X.append(chrg)
            else:
                no_sl=sl_x*2
                chrg=(file_5['fwd_e_fixed'][0]+file_5['rto_e_fixed'][0])+((no_sl-1)*(file_5['fwd_e_additional'][0]+file_5['rto_e_additional'][0]))
                CHRG_X.append(chrg)
             
    # Charges Billed by Courier Company (Rs.)
    chrg_c=file_4['Billing Amount (Rs.)'][i_4]
    CHRG_C.append(chrg_c)
    
    # Difference Between Expected Charges and Billed Charges (Rs.)
    diff=chrg-chrg_c
    DIFF.append(diff)


In [5]:
out=[AWB,AWB,TW_X,SLAB_X,TW_C,SLAB_C,ZONE_X,ZONE_C,CHRG_X,CHRG_C,DIFF]
for i,j in zip(file_out_2.columns,out):
    if i=='Order ID':
        pass
    else:
        file_output[i]=j


In [6]:
Output_Data_1=file_output
Output_Data_1

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.220,0.5,2.92,3.0,B,B,33.0,174.5,-141.5
1,2001806226,1091117222065,0.480,0.5,0.68,1.0,D,D,45.4,90.2,-44.8
2,2001806229,1091117222080,0.500,0.5,0.71,1.0,D,D,45.4,90.2,-44.8
3,2001806232,1091117222124,1.302,1.5,1.30,1.5,D,D,135.0,135.0,0.0
4,2001806233,1091117222135,0.245,0.5,0.78,1.0,B,B,33.0,61.3,-28.3
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,0.5,0.50,0.5,B,D,33.0,45.4,-12.4
120,2001822466,1091121305541,1.376,1.5,1.10,1.5,B,D,89.6,135.0,-45.4
121,2001823564,1091121666133,0.672,1.0,0.70,1.0,D,D,176.3,172.8,3.5
122,2001825261,1091121981575,1.557,2.0,1.60,2.0,D,D,355.5,345.0,10.5


### summary table (Output_Data_2)

In [7]:
S_TABLE={'Type of charges':['Total orders where X has been correctly charged','Total Orders where X has been overcharged','Total Orders where X has been undercharged']}
S_TABLE=pd.DataFrame(S_TABLE)

In [8]:
i_f_0=np.where(file_output['Difference Between Expected Charges and Billed Charges (Rs.)']==0)
i_f_n=np.where(file_output['Difference Between Expected Charges and Billed Charges (Rs.)']<0)
i_f_p=np.where(file_output['Difference Between Expected Charges and Billed Charges (Rs.)']>0)
i_f=[i_f_0,i_f_n,i_f_p]

CH=0
Co_t=[]
Am_t=[]

for i in i_f:
    count=len(i[0])
    Co_t.append(count)
    for j in i[0]:
        ch=file_output['Charges Billed by Courier Company (Rs.) '][j]
        CH=CH+ch
    Am_t.append(CH)

In [9]:
S_TABLE['Count']=Co_t
S_TABLE['Amount (Rs.)']=Am_t
Output_Data_2=S_TABLE

In [10]:
Output_Data_2

,Type of charges,Count,Amount (Rs.)
0,Total orders where X has been correctly charged,15,1207.3
1,Total Orders where X has been overcharged,85,10189.4
2,Total Orders where X has been undercharged,24,13648.2


In [14]:
'''
import xlsxwriter

excel_writer = pd.ExcelWriter(r'C:\Users\Parimal\Desktop\Cointab_DA-Challenge_Output_(Raviraj_Wagh).xlsx', engine='xlsxwriter')
Output_Data_2.to_excel(excel_writer, sheet_name='Summary_table',index=False)
Output_Data_1.to_excel(excel_writer, sheet_name='order_level_calculation',index=False)
excel_writer.save()
'''

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 54-55: truncated \UXXXXXXXX escape (Temp/ipykernel_18644/2676280583.py, line 8)

In [15]:
Output_Data_1['Difference Between Expected Charges and Billed Charges (Rs.)'].sum()

-3908.100000000001